
<img src="https://bloqueneon.uniandes.edu.co/content/enforced/52233-202120_ISIS4219_01/Laboratorios.png?_&d2lSessionVal=guLBE9l4cI9Tip6PM5JSzKgp6&_&d2lSessionVal=iDtWjzbHb92iC3sVmHUqajJ2k&_&d2lSessionVal=g2xptlvZ5GrV3W5yGlhIgeBnN" ><br>
# Machine Learning Techniques - ISIS4219

Segundo Semestre - 2021




## Integrantes


*    Laura Natalia González García.
*    Nicolas Gustavo Gaitán Gómez.
*    David Camilo Bonilla Verdugo.




## **Problema 1**

Utilice el conjunto de datos Fashion-MNIST para construir un clasificador de imágenes de productos. Para la construcción del modelo utilice los tres esquemas que se describen a continuación y compare los resultados:
<ol type="a">
  <li>Entrenamiento de un perceptrón multicapa.</li>
  <li>Entrenamiento de un encoder para realizar una reducción de la dimensionalidad. Sobre el nuevo conjunto de características construya el modelo de clasificación con un perceptrón multicapa.</li>
  <li>Entrenamiento de un denoising autoendoder para preentrenar una red profunda. Reuse las primeras capas de este autoencoder para construir un perceptron multicapa (con solo el 10% de los datos).</li>
</ol>

Para los puntos b) y c) compruebe, y muestre con ejemplos, que las imágenes están bien reconstruidas.

## **Problema 2**

Para los bancos es sumamente importante reconocer transacciones fraudulentas con el objetivo de que los clientes no paguen por cosas que no compraron. Con respecto a lo anterior, se recogieron datos de más de 200 mil transacciones con un porcentaje de 0.172% de transacciones fraudulentas. Dada la baja cantidad de fraudes, se le pidió a ud que creara un modelo de inteligencia artificial para poder detectar este tipo de anomalías.

**Fuente de Datos**: https://www.kaggle.com/mlg-ulb/creditcardfraud


# 1. Importe de librerias

In [69]:
import pandas as pd
import numpy as np
import sys
import pathlib
import matplotlib.pyplot as plt
%matplotlib inline
#from pandas_profiling import ProfileReport
from sklearn.utils import class_weight
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, make_scorer, f1_score
from sklearn.ensemble import IsolationForest
from sklearn import model_selection
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
print('Tensorflow version:', tf.__version__)
import dill

Tensorflow version: 2.6.1


In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)


Num GPUs Available:  1
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


# PROBLEMA 1

# PROBLEMA 2

# 2.b Carga y preparacion de datos

In [5]:
import os
os.chdir("/home/ngaitan55/Documents/maestria/primer_semestre/machine_learning_techniques/MLTechniques/Lab4")
transaction_data = pd.read_csv("creditcard.csv")
transaction_data.head()
transaction_data.describe()
transaction_data.shape

(284807, 31)

In [63]:
transaction_data.dropna(inplace=True)

X, y = transaction_data.loc[:,:'Amount'], transaction_data['Class']
print(np.unique(y, return_counts=True))

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.2, random_state=33)

print(np.unique(y_train, return_counts=True))
X_train

(array([0, 1]), array([284315,    492]))
(array([0, 1]), array([227451,    394]))


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
133959,80589.0,-1.029749,-0.886750,1.525456,0.018604,-0.030239,1.315017,0.029963,0.457548,0.213728,...,0.753092,0.052352,-0.093859,0.434683,-1.073242,-0.449460,1.101048,0.025587,0.136574,215.41
168762,119374.0,-0.506945,1.515877,0.898573,1.836139,1.660120,0.173390,1.390980,0.020182,-1.741904,...,-0.314393,-0.200440,-0.749549,-0.017803,0.381443,-0.325797,-0.616059,0.092138,0.177304,5.92
221513,142592.0,2.032899,-0.183798,-1.339760,-0.134025,0.332333,0.349340,-0.579505,0.188176,1.044014,...,-0.081864,-0.282935,-0.711988,0.255920,-0.476301,-0.360309,-0.271169,0.015571,-0.022352,7.32
183317,125744.0,-1.535875,-0.938860,1.348134,0.093588,0.944312,1.579554,-0.320655,0.715160,-1.486529,...,0.199371,-0.119325,-0.143056,0.167991,-1.732740,0.454227,-0.239148,0.328791,0.142486,148.00
115149,73765.0,-7.116875,3.220095,-4.938940,-1.127779,-0.866639,3.007076,-1.425167,3.593258,1.272497,...,1.005250,-0.737171,-1.460917,-0.546977,0.969683,0.580949,-0.019334,-0.352561,-0.625335,89.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76303,56504.0,1.157378,0.013319,0.982792,0.607458,-0.646507,-0.185203,-0.414174,0.042867,0.311721,...,-0.042569,-0.004022,0.062903,0.084545,0.111306,0.123145,0.276180,0.018452,0.027380,15.88
87713,61803.0,1.069284,-0.079006,1.304495,1.236450,-0.861687,0.148753,-0.567789,0.175798,0.482559,...,-0.017652,0.013422,0.208237,-0.047989,0.354829,0.409745,-0.431516,0.071060,0.033825,30.74
251820,155513.0,-2.972780,-2.403812,1.064995,-0.545382,2.368264,0.417074,-0.677048,0.661271,0.285535,...,-0.185828,0.184630,1.138684,1.511208,-1.615795,0.164760,0.849735,0.419242,0.110403,50.00
5721,6019.0,-2.414415,0.019077,2.470609,3.467957,-1.032963,1.483254,0.296172,-1.801313,1.008500,...,-1.067717,1.234428,0.134437,0.721739,0.347498,0.156281,0.381950,0.025110,0.062728,250.80


Prueba I con algoritmo Isolation forest

Modelo base

In [18]:
cont = sum(y_train == 1)/len(y_train)
print("Contamination ", cont)
iforest = IsolationForest(n_estimators = 500, contamination = cont, verbose = 1)
iforest.fit(X_train)
outlier_decision = iforest.predict(X_train)

Contamination  0.001729245759178389


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.1s finished


In [19]:
y_pred = [1 if pred == -1 else 0 for pred in outlier_decision]
report_base_IF = classification_report(y_train, y_pred)
print("Test Report para Modelo base IF\n", report_base_IF)
print("Confusion Matrix para Modelo base IF\n", confusion_matrix(y_train, y_pred))


Test Report para Modelo base IF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.26      0.26      0.26       394

    accuracy                           1.00    227845
   macro avg       0.63      0.63      0.63    227845
weighted avg       1.00      1.00      1.00    227845

Confusion Matrix para Modelo base IF
 [[227159    292]
 [   292    102]]


Modelo con optimizacion de hiperparametros

In [20]:
skf = StratifiedKFold(n_splits=3)
folds = list(skf.split(X_train, y_train))

In [21]:
clf = IsolationForest(random_state=47, verbose = 100, contamination = cont)

y_conv = y_train.astype(np.int8)
y_conv[y_conv==1] = -1
y_conv[y_conv==0] = 1

param_grid = {'n_estimators': [100, 250,  500], 
              'max_samples': list(range(100, 250, 500)),
              'max_features': [5,10,15]
              }

f1sc = make_scorer(f1_score)
grid_search_if = GridSearchCV(clf, 
                            param_grid,
                            scoring=f1sc, 
                            return_train_score=True,
                            cv = folds)
grid_search_if.fit(X_train, y_conv)
print(grid_search_if.best_params_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 100 for this parallel run (total 100)...
Building estimator 2 of 100 for this parallel run (total 100)...
Building estimator 3 of 100 for this parallel run (total 100)...
Building estimator 4 of 100 for this parallel run (total 100)...
Building estimator 5 of 100 for this parallel run (total 100)...
Building estimator 6 of 100 for this parallel run (total 100)...
Building estimator 7 of 100 for this parallel run (total 100)...
Building estimator 8 of 100 for this parallel run (total 100)...
Building estimator 9 of 100 for this parallel run (total 100)...
Building estimator 10 of 100 for this parallel run (total 100)...
Building estimator 11 of 100 for this parallel run (total 100)...
Building estimator 12 of 100 for this parallel run (total 100)...
Building estimator 13 of 100 for this parallel run (total 100)...
Building estimator 14 of 100 for this parallel run (total 100)...
Buil

Building estimator 57 of 100 for this parallel run (total 100)...
Building estimator 58 of 100 for this parallel run (total 100)...
Building estimator 59 of 100 for this parallel run (total 100)...
Building estimator 60 of 100 for this parallel run (total 100)...
Building estimator 61 of 100 for this parallel run (total 100)...
Building estimator 62 of 100 for this parallel run (total 100)...
Building estimator 63 of 100 for this parallel run (total 100)...
Building estimator 64 of 100 for this parallel run (total 100)...
Building estimator 65 of 100 for this parallel run (total 100)...
Building estimator 66 of 100 for this parallel run (total 100)...
Building estimator 67 of 100 for this parallel run (total 100)...
Building estimator 68 of 100 for this parallel run (total 100)...
Building estimator 69 of 100 for this parallel run (total 100)...
Building estimator 70 of 100 for this parallel run (total 100)...
Building estimator 71 of 100 for this parallel run (total 100)...
Building e

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 250 for this parallel run (total 250)...
Building estimator 2 of 250 for this parallel run (total 250)...
Building estimator 3 of 250 for this parallel run (total 250)...
Building estimator 4 of 250 for this parallel run (total 250)...
Building estimator 5 of 250 for this parallel run (total 250)...
Building estimator 6 of 250 for this parallel run (total 250)...
Building estimator 7 of 250 for this parallel run (total 250)...
Building estimator 8 of 250 for this parallel run (total 250)...
Building estimator 9 of 250 for this parallel run (total 250)...
Building estimator 10 of 250 for this parallel run (total 250)...
Building estimator 11 of 250 for this parallel run (total 250)...
Building estimator 12 of 250 for this parallel run (total 250)...
Building estimator 13 of 250 for this parallel run (total 250)...
Building estimator 14 of 250 for this parallel run (total 250)...
Buil

Building estimator 178 of 250 for this parallel run (total 250)...
Building estimator 179 of 250 for this parallel run (total 250)...
Building estimator 180 of 250 for this parallel run (total 250)...
Building estimator 181 of 250 for this parallel run (total 250)...
Building estimator 182 of 250 for this parallel run (total 250)...
Building estimator 183 of 250 for this parallel run (total 250)...
Building estimator 184 of 250 for this parallel run (total 250)...
Building estimator 185 of 250 for this parallel run (total 250)...
Building estimator 186 of 250 for this parallel run (total 250)...
Building estimator 187 of 250 for this parallel run (total 250)...
Building estimator 188 of 250 for this parallel run (total 250)...
Building estimator 189 of 250 for this parallel run (total 250)...
Building estimator 190 of 250 for this parallel run (total 250)...
Building estimator 191 of 250 for this parallel run (total 250)...
Building estimator 192 of 250 for this parallel run (total 250

Building estimator 55 of 250 for this parallel run (total 250)...
Building estimator 56 of 250 for this parallel run (total 250)...
Building estimator 57 of 250 for this parallel run (total 250)...
Building estimator 58 of 250 for this parallel run (total 250)...
Building estimator 59 of 250 for this parallel run (total 250)...
Building estimator 60 of 250 for this parallel run (total 250)...
Building estimator 61 of 250 for this parallel run (total 250)...
Building estimator 62 of 250 for this parallel run (total 250)...
Building estimator 63 of 250 for this parallel run (total 250)...
Building estimator 64 of 250 for this parallel run (total 250)...
Building estimator 65 of 250 for this parallel run (total 250)...
Building estimator 66 of 250 for this parallel run (total 250)...
Building estimator 67 of 250 for this parallel run (total 250)...
Building estimator 68 of 250 for this parallel run (total 250)...
Building estimator 69 of 250 for this parallel run (total 250)...
Building e

...

Building estimator 425 of 500 for this parallel run (total 500)...
Building estimator 426 of 500 for this parallel run (total 500)...
Building estimator 427 of 500 for this parallel run (total 500)...
Building estimator 428 of 500 for this parallel run (total 500)...
Building estimator 429 of 500 for this parallel run (total 500)...
Building estimator 430 of 500 for this parallel run (total 500)...
Building estimator 431 of 500 for this parallel run (total 500)...
Building estimator 432 of 500 for this parallel run (total 500)...
Building estimator 433 of 500 for this parallel run (total 500)...
Building estimator 434 of 500 for this parallel run (total 500)...
Building estimator 435 of 500 for this parallel run (total 500)...
Building estimator 436 of 500 for this parallel run (total 500)...
Building estimator 437 of 500 for this parallel run (total 500)...
Building estimator 438 of 500 for this parallel run (total 500)...
Building estimator 439 of 500 for this parallel run (total 500

Building estimator 61 of 500 for this parallel run (total 500)...
Building estimator 62 of 500 for this parallel run (total 500)...
Building estimator 63 of 500 for this parallel run (total 500)...
Building estimator 64 of 500 for this parallel run (total 500)...
Building estimator 65 of 500 for this parallel run (total 500)...
Building estimator 66 of 500 for this parallel run (total 500)...
Building estimator 67 of 500 for this parallel run (total 500)...
Building estimator 68 of 500 for this parallel run (total 500)...
Building estimator 69 of 500 for this parallel run (total 500)...
Building estimator 70 of 500 for this parallel run (total 500)...
Building estimator 71 of 500 for this parallel run (total 500)...
Building estimator 72 of 500 for this parallel run (total 500)...
Building estimator 73 of 500 for this parallel run (total 500)...
Building estimator 74 of 500 for this parallel run (total 500)...
Building estimator 75 of 500 for this parallel run (total 500)...
Building e

Building estimator 206 of 500 for this parallel run (total 500)...
Building estimator 207 of 500 for this parallel run (total 500)...
Building estimator 208 of 500 for this parallel run (total 500)...
Building estimator 209 of 500 for this parallel run (total 500)...
Building estimator 210 of 500 for this parallel run (total 500)...
Building estimator 211 of 500 for this parallel run (total 500)...
Building estimator 212 of 500 for this parallel run (total 500)...
Building estimator 213 of 500 for this parallel run (total 500)...
Building estimator 214 of 500 for this parallel run (total 500)...
Building estimator 215 of 500 for this parallel run (total 500)...
Building estimator 216 of 500 for this parallel run (total 500)...
Building estimator 217 of 500 for this parallel run (total 500)...
Building estimator 218 of 500 for this parallel run (total 500)...
Building estimator 219 of 500 for this parallel run (total 500)...
Building estimator 220 of 500 for this parallel run (total 500

Building estimator 354 of 500 for this parallel run (total 500)...
Building estimator 355 of 500 for this parallel run (total 500)...
Building estimator 356 of 500 for this parallel run (total 500)...
Building estimator 357 of 500 for this parallel run (total 500)...
Building estimator 358 of 500 for this parallel run (total 500)...
Building estimator 359 of 500 for this parallel run (total 500)...
Building estimator 360 of 500 for this parallel run (total 500)...
Building estimator 361 of 500 for this parallel run (total 500)...
Building estimator 362 of 500 for this parallel run (total 500)...
Building estimator 363 of 500 for this parallel run (total 500)...
Building estimator 364 of 500 for this parallel run (total 500)...
Building estimator 365 of 500 for this parallel run (total 500)...
Building estimator 366 of 500 for this parallel run (total 500)...
Building estimator 367 of 500 for this parallel run (total 500)...
Building estimator 368 of 500 for this parallel run (total 500

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 100 for this parallel run (total 100)...
Building estimator 2 of 100 for this parallel run (total 100)...
Building estimator 3 of 100 for this parallel run (total 100)...
Building estimator 4 of 100 for this parallel run (total 100)...
Building estimator 5 of 100 for this parallel run (total 100)...
Building estimator 6 of 100 for this parallel run (total 100)...
Building estimator 7 of 100 for this parallel run (total 100)...
Building estimator 8 of 100 for this parallel run (total 100)...
Building estimator 9 of 100 for this parallel run (total 100)...
Building estimator 10 of 100 for this parallel run (total 100)...
Building estimator 11 of 100 for this parallel run (total 100)...
Building estimator 12 of 100 for this parallel run (total 100)...
Building estimator 13 of 100 for this parallel run (total 100)...
Building estimator 14 of 100 for this parallel run (total 100)...
Buil

In [26]:
best_model_if = grid_search_if.best_estimator_
outlier_pred_if_best_model = best_model_if.predict(X_train)
Y_pred_if_best_model = [1 if pred == -1 else 0 for pred in outlier_pred_if_best_model]
report_best_IF = classification_report(y_train, Y_pred_if_best_model)
print("Test Report para mejor modelo IF\n", report_best_IF)
print("Confusion Matrix para mejor modelo IF\n", confusion_matrix(y_train, Y_pred_if_best_model))

Test Report para mejor modelo IF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    227451
           1       0.34      0.34      0.34       394

    accuracy                           1.00    227845
   macro avg       0.67      0.67      0.67    227845
weighted avg       1.00      1.00      1.00    227845

Confusion Matrix para mejor modelo IF
 [[227192    259]
 [   259    135]]


Modelo con remuestreo de la clase minoritaria y isolation forest

In [64]:
for_sampling = X_train.copy()
for_sampling.loc[:,'Class'] = y_train 
majority = for_sampling[for_sampling.Class==0]
minority = for_sampling[for_sampling.Class==1]
minority_upsampled = resample(minority, 
                                 replace=True,     
                                 n_samples=len(majority),   
                                 random_state=123)
upsampled = pd.concat([majority, minority_upsampled])
print(upsampled.Class.value_counts())
X_train_upsampled, y_train_upsampled = upsampled.loc[:,:'Amount'], upsampled['Class']
print(np.unique(y_train_upsampled, return_counts=True))


0    227451
1    227451
Name: Class, dtype: int64
(array([0, 1]), array([227451, 227451]))


In [35]:
cont_ups = sum(y_train_upsampled == 1)/len(y_train_upsampled)
print("Contamination ", cont_ups)
iforest_ups = IsolationForest(n_estimators = 500, contamination = cont_ups, verbose = 1)
iforest_ups.fit(X_train_upsampled)
outlier_decision_ups = iforest.predict(X_train_upsampled)

y_pred_ups = [1 if pred == -1 else 0 for pred in outlier_decision_ups]
report_base_IF_ups = classification_report(y_train_upsampled, y_pred_ups)
print("Test Report para Modelo base IF\n", report_base_IF_ups)
print("Confusion Matrix para Modelo base IF\n", confusion_matrix(y_train_upsampled, y_pred_ups))

Contamination  0.5


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.5s finished


Test Report para Modelo base IF
               precision    recall  f1-score   support

           0       0.57      1.00      0.73    227451
           1       1.00      0.26      0.41    227451

    accuracy                           0.63    454902
   macro avg       0.78      0.63      0.57    454902
weighted avg       0.78      0.63      0.57    454902

Confusion Matrix para Modelo base IF
 [[227159    292]
 [168918  58533]]


In [36]:
skf = StratifiedKFold(n_splits=3)
folds = list(skf.split(X_train_upsampled, y_train_upsampled))

clf = IsolationForest(random_state=47, verbose = 100, contamination = cont_ups)

y_conv_ups = y_train_upsampled.astype(np.int8)
y_conv_ups[y_conv_ups==1] = -1
y_conv_ups[y_conv_ups==0] = 1

param_grid = {'n_estimators': [100, 250,  500], 
              'max_samples': list(range(100, 250, 500)),
              'max_features': [5,10,15]
              }

f1sc = make_scorer(f1_score)
grid_search_if = GridSearchCV(clf, 
                            param_grid,
                            scoring=f1sc, 
                            return_train_score=True,
                            cv = folds)
grid_search_if.fit(X_train_upsampled, y_conv_ups)
print(grid_search_if.best_params_)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
Building estimator 1 of 100 for this parallel run (total 100)...
Building estimator 2 of 100 for this parallel run (total 100)...
Building estimator 3 of 100 for this parallel run (total 100)...
Building estimator 4 of 100 for this parallel run (total 100)...
Building estimator 5 of 100 for this parallel run (total 100)...
Building estimator 6 of 100 for this parallel run (total 100)...
Building estimator 7 of 100 for this parallel run (total 100)...
Building estimator 8 of 100 for this parallel run (total 100)...
Building estimator 9 of 100 for this parallel run (total 100)...
Building estimator 10 of 100 for this parallel run (total 100)...
Building estimator 11 of 100 for this parallel run (total 100)...
Building estimator 12 of 100 for this parallel run (total 100)...
Building estimator 13 of 100 for this parallel run (total 100)...
Building estimator 14 of 100 for this parallel run (total 100)...
Buil

Building estimator 36 of 100 for this parallel run (total 100)...
Building estimator 37 of 100 for this parallel run (total 100)...
Building estimator 38 of 100 for this parallel run (total 100)...
Building estimator 39 of 100 for this parallel run (total 100)...
Building estimator 40 of 100 for this parallel run (total 100)...
Building estimator 41 of 100 for this parallel run (total 100)...
Building estimator 42 of 100 for this parallel run (total 100)...
Building estimator 43 of 100 for this parallel run (total 100)...
Building estimator 44 of 100 for this parallel run (total 100)...
Building estimator 45 of 100 for this parallel run (total 100)...
Building estimator 46 of 100 for this parallel run (total 100)...
Building estimator 47 of 100 for this parallel run (total 100)...
Building estimator 48 of 100 for this parallel run (total 100)...
Building estimator 49 of 100 for this parallel run (total 100)...
Building estimator 50 of 100 for this parallel run (total 100)...
Building e

Building estimator 70 of 100 for this parallel run (total 100)...
Building estimator 71 of 100 for this parallel run (total 100)...
Building estimator 72 of 100 for this parallel run (total 100)...
Building estimator 73 of 100 for this parallel run (total 100)...
Building estimator 74 of 100 for this parallel run (total 100)...
Building estimator 75 of 100 for this parallel run (total 100)...
Building estimator 76 of 100 for this parallel run (total 100)...
Building estimator 77 of 100 for this parallel run (total 100)...
Building estimator 78 of 100 for this parallel run (total 100)...
Building estimator 79 of 100 for this parallel run (total 100)...
Building estimator 80 of 100 for this parallel run (total 100)...
Building estimator 81 of 100 for this parallel run (total 100)...
Building estimator 82 of 100 for this parallel run (total 100)...
Building estimator 83 of 100 for this parallel run (total 100)...
Building estimator 84 of 100 for this parallel run (total 100)...
Building e

Building estimator 97 of 250 for this parallel run (total 250)...
Building estimator 98 of 250 for this parallel run (total 250)...
Building estimator 99 of 250 for this parallel run (total 250)...
Building estimator 100 of 250 for this parallel run (total 250)...
Building estimator 101 of 250 for this parallel run (total 250)...
Building estimator 102 of 250 for this parallel run (total 250)...
Building estimator 103 of 250 for this parallel run (total 250)...
Building estimator 104 of 250 for this parallel run (total 250)...
Building estimator 105 of 250 for this parallel run (total 250)...
Building estimator 106 of 250 for this parallel run (total 250)...
Building estimator 107 of 250 for this parallel run (total 250)...
Building estimator 108 of 250 for this parallel run (total 250)...
Building estimator 109 of 250 for this parallel run (total 250)...
Building estimator 110 of 250 for this parallel run (total 250)...
Building estimator 111 of 250 for this parallel run (total 250)..

Building estimator 233 of 250 for this parallel run (total 250)...
Building estimator 234 of 250 for this parallel run (total 250)...
Building estimator 235 of 250 for this parallel run (total 250)...
Building estimator 236 of 250 for this parallel run (total 250)...
Building estimator 237 of 250 for this parallel run (total 250)...
Building estimator 238 of 250 for this parallel run (total 250)...
Building estimator 239 of 250 for this parallel run (total 250)...
Building estimator 240 of 250 for this parallel run (total 250)...
Building estimator 241 of 250 for this parallel run (total 250)...
Building estimator 242 of 250 for this parallel run (total 250)...
Building estimator 243 of 250 for this parallel run (total 250)...
Building estimator 244 of 250 for this parallel run (total 250)...
Building estimator 245 of 250 for this parallel run (total 250)...
Building estimator 246 of 250 for this parallel run (total 250)...
Building estimator 247 of 250 for this parallel run (total 250

...

Building estimator 436 of 500 for this parallel run (total 500)...
Building estimator 437 of 500 for this parallel run (total 500)...
Building estimator 438 of 500 for this parallel run (total 500)...
Building estimator 439 of 500 for this parallel run (total 500)...
Building estimator 440 of 500 for this parallel run (total 500)...
Building estimator 441 of 500 for this parallel run (total 500)...
Building estimator 442 of 500 for this parallel run (total 500)...
Building estimator 443 of 500 for this parallel run (total 500)...
Building estimator 444 of 500 for this parallel run (total 500)...
Building estimator 445 of 500 for this parallel run (total 500)...
Building estimator 446 of 500 for this parallel run (total 500)...
Building estimator 447 of 500 for this parallel run (total 500)...
Building estimator 448 of 500 for this parallel run (total 500)...
Building estimator 449 of 500 for this parallel run (total 500)...
Building estimator 450 of 500 for this parallel run (total 500

Building estimator 82 of 500 for this parallel run (total 500)...
Building estimator 83 of 500 for this parallel run (total 500)...
Building estimator 84 of 500 for this parallel run (total 500)...
Building estimator 85 of 500 for this parallel run (total 500)...
Building estimator 86 of 500 for this parallel run (total 500)...
Building estimator 87 of 500 for this parallel run (total 500)...
Building estimator 88 of 500 for this parallel run (total 500)...
Building estimator 89 of 500 for this parallel run (total 500)...
Building estimator 90 of 500 for this parallel run (total 500)...
Building estimator 91 of 500 for this parallel run (total 500)...
Building estimator 92 of 500 for this parallel run (total 500)...
Building estimator 93 of 500 for this parallel run (total 500)...
Building estimator 94 of 500 for this parallel run (total 500)...
Building estimator 95 of 500 for this parallel run (total 500)...
Building estimator 96 of 500 for this parallel run (total 500)...
Building e

Building estimator 217 of 500 for this parallel run (total 500)...
Building estimator 218 of 500 for this parallel run (total 500)...
Building estimator 219 of 500 for this parallel run (total 500)...
Building estimator 220 of 500 for this parallel run (total 500)...
Building estimator 221 of 500 for this parallel run (total 500)...
Building estimator 222 of 500 for this parallel run (total 500)...
Building estimator 223 of 500 for this parallel run (total 500)...
Building estimator 224 of 500 for this parallel run (total 500)...
Building estimator 225 of 500 for this parallel run (total 500)...
Building estimator 226 of 500 for this parallel run (total 500)...
Building estimator 227 of 500 for this parallel run (total 500)...
Building estimator 228 of 500 for this parallel run (total 500)...
Building estimator 229 of 500 for this parallel run (total 500)...
Building estimator 230 of 500 for this parallel run (total 500)...
Building estimator 231 of 500 for this parallel run (total 500

Building estimator 351 of 500 for this parallel run (total 500)...
Building estimator 352 of 500 for this parallel run (total 500)...
Building estimator 353 of 500 for this parallel run (total 500)...
Building estimator 354 of 500 for this parallel run (total 500)...
Building estimator 355 of 500 for this parallel run (total 500)...
Building estimator 356 of 500 for this parallel run (total 500)...
Building estimator 357 of 500 for this parallel run (total 500)...
Building estimator 358 of 500 for this parallel run (total 500)...
Building estimator 359 of 500 for this parallel run (total 500)...
Building estimator 360 of 500 for this parallel run (total 500)...
Building estimator 361 of 500 for this parallel run (total 500)...
Building estimator 362 of 500 for this parallel run (total 500)...
Building estimator 363 of 500 for this parallel run (total 500)...
Building estimator 364 of 500 for this parallel run (total 500)...
Building estimator 365 of 500 for this parallel run (total 500

Building estimator 486 of 500 for this parallel run (total 500)...
Building estimator 487 of 500 for this parallel run (total 500)...
Building estimator 488 of 500 for this parallel run (total 500)...
Building estimator 489 of 500 for this parallel run (total 500)...
Building estimator 490 of 500 for this parallel run (total 500)...
Building estimator 491 of 500 for this parallel run (total 500)...
Building estimator 492 of 500 for this parallel run (total 500)...
Building estimator 493 of 500 for this parallel run (total 500)...
Building estimator 494 of 500 for this parallel run (total 500)...
Building estimator 495 of 500 for this parallel run (total 500)...
Building estimator 496 of 500 for this parallel run (total 500)...
Building estimator 497 of 500 for this parallel run (total 500)...
Building estimator 498 of 500 for this parallel run (total 500)...
Building estimator 499 of 500 for this parallel run (total 500)...
Building estimator 500 of 500 for this parallel run (total 500

Prueba I con algoritmo One class SVM

Modelo base


In [73]:
ratio = sum(y_train==1)/len(y_train)
X_train = X_train.drop('Time', 1)
scaler = StandardScaler().fit(X_train)
X_train_norm = scaler.transform(X_train)
OCSVM = OneClassSVM(gamma='scale', nu=ratio)
OCSVM.fit(X_train_norm)
outlier_decision_OSVM = OCSVM.predict(X_train_norm)
y_pred_svm = [1 if pred == -1 else 0 for pred in outlier_decision_OSVM]
report_base_OSVM = classification_report(y_train, y_pred_svm)
print("Test Report para Modelo base IF\n", report_base_OSVM)
print("Confusion Matrix para Modelo base IF\n", confusion_matrix(y_train, y_pred_svm))

<ipython-input-73-e8bc5edeb4ad>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train = X_train.drop('Time', 1)


Test Report para Modelo base IF
               precision    recall  f1-score   support

           0       1.00      0.99      1.00    227451
           1       0.08      0.24      0.11       394

    accuracy                           0.99    227845
   macro avg       0.54      0.62      0.56    227845
weighted avg       1.00      0.99      1.00    227845

Confusion Matrix para Modelo base IF
 [[226282   1169]
 [   299     95]]


In [49]:
X_train.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
133959,80589.0,-1.029749,-0.886750,1.525456,0.018604,-0.030239,1.315017,0.029963,0.457548,0.213728,...,0.052352,-0.093859,0.434683,-1.073242,-0.449460,1.101048,0.025587,0.136574,215.41,0
168762,119374.0,-0.506945,1.515877,0.898573,1.836139,1.660120,0.173390,1.390980,0.020182,-1.741904,...,-0.200440,-0.749549,-0.017803,0.381443,-0.325797,-0.616059,0.092138,0.177304,5.92,0
221513,142592.0,2.032899,-0.183798,-1.339760,-0.134025,0.332333,0.349340,-0.579505,0.188176,1.044014,...,-0.282935,-0.711988,0.255920,-0.476301,-0.360309,-0.271169,0.015571,-0.022352,7.32,0
183317,125744.0,-1.535875,-0.938860,1.348134,0.093588,0.944312,1.579554,-0.320655,0.715160,-1.486529,...,-0.119325,-0.143056,0.167991,-1.732740,0.454227,-0.239148,0.328791,0.142486,148.00,0
115149,73765.0,-7.116875,3.220095,-4.938940,-1.127779,-0.866639,3.007076,-1.425167,3.593258,1.272497,...,-0.737171,-1.460917,-0.546977,0.969683,0.580949,-0.019334,-0.352561,-0.625335,89.99,0


In [ ]:

y_conv_svm = y_train.astype(np.int8)
y_conv_svm[y_conv_svm==1] = -1
y_conv_svm[y_conv_svm==0] = 1

In [74]:
dill.dump_session('Tarea4_env.db')

In [ ]:
dill.load_session('Tarea4_env.db')